In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from tensorflow.keras import datasets, layers, models, regularizers, initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.io import imshow
from skimage.transform import rotate
from skimage.filters.edges import convolve

from skimage.io import imshow, imread
from skimage.transform import resize
from skimage.color import rgb2gray

import dill as pickle

np.random.seed(33)

In [3]:
df_test = pd.read_csv('data/test_labels.csv')
df_train = pd.read_csv('data/train_labels.csv')

bw_loaded = np.load('data/bw_images.npz')
X_train= bw_loaded['a']
X_test = bw_loaded['b']

X_test = X_test.reshape(-1,80,60,1)
X_train = X_train.reshape(-1,80,60,1)

In [4]:
# from model_functions import pick_ylabels, multi_index_counts, test_counts_by_cat, train_counts_by_cat, category_codes, class_weights

In [5]:
def pick_ylabels(column):
    y_train = df_train[column].copy().astype('category').cat.codes
    y_test = df_test[column].copy().astype('category').cat.codes
    return (y_train.values, y_test.values)
def multi_index_counts(col, col2):
    counts = df_test.groupby([col, col2]).count().id
    return counts

def category_codes(column):
    _, ytest = pick_ylabels(column)
    cat_codes = {}
    cat_code_list = []
    for i in range(len(df_test[column].value_counts().index)):
        s = i
        t = df_test[column].value_counts().index[i]
        cat_codes[s] = t
#         cat_code_list.append([s, t])

#     for key in sorted(cat_codes):
#         print("%s: %s" % (key, cat_codes[key]))
    return cat_codes

def test_counts_by_cat(column):
    _, ytest = pick_ylabels(column)
    test_counts_dict = {}
    test_counts = []
    for i in range(len(df_test[column].value_counts().index)):
        s = df_test[column].value_counts().values[i]
        t = df_test[column].value_counts().index[i]
        test_counts_dict[t] = s
        test_counts.append([t, s])
#     for i in sorted(test_counts):
#         print("%s: %s" % (test_counts[0], test_counts[1]))
    return test_counts

def train_counts_by_cat(column):
    y_train = pick_ylabels(column)
    train_counts_dict = {}
    train_counts = []
    for i in range(len(df_test[column].value_counts().index)):
        s = df_train[column].value_counts().values[i]
        t = df_train[column].value_counts().index[i]
        train_counts_dict[t] = s
        train_counts.append([t, s])
#     for i in sorted(test_counts):
#         print("%s: %s" % (test_counts[0], test_counts[1]))
    return test_counts

def class_weights(column):
    train_counts = train_counts_by_cat(column)
    counts_list = []
    ratio_list = []
    ratio_dict = {}
    for x in test_counts:
        counts_list.append(x[1])
    #print(counts_list)
    z = sum(counts_list)
    for x in counts_list:
        if np.round(x/z, 3) ==0:
            ratio_list.append(.001)
        else: 
            ratio_list.append(np.round(x/z, 2))
    for k, v in enumerate(ratio_list):
        ratio_dict[k] = v
    return ratio_dict

def model_results(model, column):
    yhat = model.predict(X_test)
    _ , ytest = pick_ylabels(column)
    accuracy = accuracy_score(ytest, yhat)
    print( 'Accuracy Score: ', accuracy )
    recall = recall_score(ytest, yhat, average='weighted')
    print( 'Recall Score: ', recall)
    y_proba = model.predict_proba(X_test)
    wrong_id_list = []
    pred_cat_list = []
    real_cat_list = []
    for row_idx in range(len(ytest)):
        if ytest[row_idx]!=yhat[row_idx]:
            wrong_id_list.append(row_idx)
            pred_cat_list.append(yhat[row_idx])
            real_cat_list.append(ytest[row_idx])

    arr = np.array([ real_cat_list, pred_cat_list])
    arr = arr.transpose()
    wrong_df = pd.DataFrame( arr, index= wrong_id_list, columns = ['actual', 'predicted'] )
    return (accuracy, recall, wrong_df)

In [6]:
y_train0, y_test0 = pick_ylabels('masterCategory')

In [7]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train0)
y_test = to_categorical(y_test0)

In [8]:
y_train.shape

(25439, 4)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

In [10]:
cnn1 = models.Sequential()



In [11]:
cnn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
#MaxPool reduces dimensionality of each feature
cnn1.add(layers.MaxPooling2D((2, 2)))
#Dropout to reduce overfitting
cnn1.add(Dropout(0.2))

W0730 17:06:49.828440 140568897226560 deprecation.py:506] From /home/ubuntu/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
cnn1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 78, 58, 32)        320       
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


In [13]:
#flatten 3D outputs to 1D
cnn1.add(Flatten())
#there are 4 main categories, and softmax will calculate probabilities of each class
cnn1.add(Dense(4, activation='softmax'))


In [14]:
cnn1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
cnn1.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)

Train on 25439 samples, validate on 6440 samples
Epoch 1/10
25439/25439 [==============================] - 17s 659us/sample - loss: 67.5887 - acc: 0.8980 - val_loss: 0.6874 - val_acc: 0.9547
Epoch 2/10
25439/25439 [==============================] - 14s 539us/sample - loss: 0.4225 - acc: 0.9689 - val_loss: 0.6469 - val_acc: 0.9648
Epoch 3/10
25439/25439 [==============================] - 14s 544us/sample - loss: 0.4069 - acc: 0.9671 - val_loss: 0.6649 - val_acc: 0.9512
Epoch 4/10
25439/25439 [==============================] - 13s 526us/sample - loss: 0.2014 - acc: 0.9790 - val_loss: 0.6316 - val_acc: 0.9590
Epoch 5/10
25439/25439 [==============================] - 14s 542us/sample - loss: 0.1618 - acc: 0.9815 - val_loss: 0.6002 - val_acc: 0.9609
Epoch 6/10
25439/25439 [==============================] - 14s 539us/sample - loss: 0.1191 - acc: 0.9841 - val_loss: 0.5987 - val_acc: 0.9590
Epoch 7/10
25439/25439 [==============================] - 14s 532us/sample - loss: 0.1401 - acc: 0.9829 

In [16]:
# save the fitted model
model_json = cnn1.to_json()
with open("cnn1.json", "w") as json_file:
    json_file.write(model_json)

In [17]:
y_true = y_test0

In [18]:
# get the predictions for the test data
predicted_classes = cnn1.predict_classes(X_test)

# get the indices to be plotted
correct = np.nonzero(predicted_classes==y_test0)[0]
incorrect = np.nonzero(predicted_classes!=y_test0)[0]

In [19]:
predicted_classes

array([1, 2, 1, ..., 1, 1, 1])

In [20]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(4)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.88      0.92      0.90      1626
     Class 1       0.97      0.95      0.96      2603
     Class 2       0.98      0.96      0.97      1826
     Class 3       0.80      0.84      0.82       385

    accuracy                           0.94      6440
   macro avg       0.91      0.92      0.91      6440
weighted avg       0.94      0.94      0.94      6440



In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=7, width_shift_range=0.05, shear_range=0.05,
                               height_shift_range=0.05, zoom_range=0.1)
datagen = gen.flow(X_train, y_train)

In [28]:
test1 = cnn1.fit_generator(datagen, epochs = 50, use_multiprocessing=True)

Epoch 1/50
795/795 [==============================] - 20s 26ms/step - loss: 0.8930 - acc: 0.6361
Epoch 2/50
795/795 [==============================] - 20s 25ms/step - loss: 0.7994 - acc: 0.6873
Epoch 3/50
795/795 [==============================] - 20s 25ms/step - loss: 0.7698 - acc: 0.7019
Epoch 4/50
795/795 [==============================] - 21s 26ms/step - loss: 1.1720 - acc: 0.5259
Epoch 5/50
795/795 [==============================] - 20s 26ms/step - loss: 0.9353 - acc: 0.6340
Epoch 6/50
795/795 [==============================] - 20s 25ms/step - loss: 0.8116 - acc: 0.6909
Epoch 7/50
795/795 [==============================] - 20s 25ms/step - loss: 0.7268 - acc: 0.7392
Epoch 8/50
795/795 [==============================] - 20s 25ms/step - loss: 1.0258 - acc: 0.6031
Epoch 9/50
795/795 [==============================] - 20s 25ms/step - loss: 0.9176 - acc: 0.6191
Epoch 10/50
795/795 [==============================] - 20s 25ms/step - loss: 0.7116 - acc: 0.7396
Epoch 11/50
795/795 [========

In [23]:
from sklearn.utils import class_weight

In [ ]:
def model_results(model, column):
    yhat = model.predict(X_test)
    _ , ytest = pick_ylabels(column)
    accuracy = accuracy_score(ytest, yhat)
    print( 'Accuracy Score: ', accuracy )
    recall = recall_score(ytest, yhat, average='weighted')
    print( 'Recall Score: ', recall)
    y_proba = model.predict_proba(X_test)
    wrong_id_list = []
    pred_cat_list = []
    real_cat_list = []
    for row_idx in range(len(ytest)):
        if ytest[row_idx]!=yhat[row_idx]:
            wrong_id_list.append(row_idx)
            pred_cat_list.append(yhat[row_idx])
            real_cat_list.append(ytest[row_idx])

    arr = np.array([ real_cat_list, pred_cat_list])
    arr = arr.transpose()
    wrong_df = pd.DataFrame( arr, index= wrong_id_list, columns = ['actual', 'predicted'] )
    return (accuracy, recall, wrong_df)